**This contains functions that will probably be used for my thesis and added to sunbather. These are drafts of the functions, not the final form**

**For Construct_parker**

In [1]:
def mu_HHe(Hefrac):
    '''
    This function takes an array of Helium abundances varying with radius and calculates the mean molecular weight. It assumes 0 ionization.
    The input parameter is temporary, it will probably be different after I incorporate crossover mass. The array should be something like [1.0e-10, 1.019e-10, 1.023e-10,......,1,9.8e-02,1.0e-01] 
    but can also be descending order.
    Only one value is returned here, not the value at each radius. mu_0 has to be a float for the pwinds_hydrogen.ion_fraction function.
    '''
    h_fraction = 1-Hefrac
    he_h_fraction = Hefrac / h_fraction
    mean_f_ion = 0.0
    mu_array = (1 + 4*he_h_fraction)/(1 + he_h_fraction + mean_f_ion)

    mu_0 = np.mean(mu_array)
    return mu_0

In [2]:
def muarray_HHe(Hefrac,f_r):
    '''
    This function takes an array of Helium abundances varying with altitude and the fraction of ionized hydrogen with planetary radius, and returns array of mean molecular weights 
    with altitude. It assumes no Helium ionization. The input parameter Hefrac is temporary, it will probably be different after I incorporate crossover mass. The array should be 
    something like [1.0e-10, 1.019e-10, 1.023e-10,......,1,9.8e-02,1.0e-01] but can also be descending order
    '''
    h_fraction = 1-Hefrac
    mu_array = ((1-h_fraction)*4.0 + h_fraction)/(h_fraction*(1+f_r)+(1-h_fraction))
    return mu_array

In [3]:
def find_fracelements(cmass):
    '''
    This returns a list of all the elements that are heavier than the crossover mass, i.e. elements that would not 
    be lifted into upper atm
    '''
    mass_dict = {'H':1.6735575e-24, 'He':6.646477e-24, 'Li':1.15e-23, 'Be':1.4965082e-23,
            'B':1.795e-23, 'C':1.9945e-23, 'N':2.3259e-23, 'O':2.6567e-23,
            'F':3.1547e-23, 'Ne':3.35092e-23, 'Na':3.817541e-23, 'Mg':4.0359e-23,
            'Al':4.48038988e-23, 'Si':4.6636e-23, 'P':5.14331418e-23, 'S':5.324e-23,
            'Cl':5.887e-23, 'Ar':6.6335e-23, 'K':6.49243e-23, 'Ca':6.6551e-23,
            'Sc':7.4651042e-23, 'Ti':7.9485e-23, 'V':8.45904e-23, 'Cr':8.63416e-23,
            'Mn':9.1226768e-23, 'Fe':9.2733e-23, 'Co':9.786087e-23, 'Ni':9.74627e-23,
            'Cu':1.0552e-22, 'Zn':1.086e-22}
    #https://stackoverflow.com/questions/18807079/selecting-elements-of-a-python-dictionary-greater-than-a-certain-value
    
    frac_list = list(k for k, v in mass_dict.items() if v > cmass)     
    return frac_list

In [4]:
#This function will probably not be used
def create_profile(ele,altarray):
    '''
    Temporary method to create abundance profile for fractionated elements. I am arbitrarily assuming that the abundance drops by 
    3 orders of magnitude over 1 Rp, then drops to 1e-40 by 8Rp.
    '''
    zdict = tools.get_abundances()
    totalele = len(altarray)
    ini_abundance = np.log10(zdict[ele])
    inter_abundance = ini_abundance-3 #abundance assumed to go down by 3 orders of magnitude over 1 Rp
    rp2 = altarray[0]*2
    elenum = np.argwhere(altarray>rp2)[0]
    abunarray1 = np.logspace(ini_abundance,inter_abundance,elenum.item())
    rem_elenum = totalele - elenum.item()
    abunarray2 = np.logspace(inter_abundance,-40,rem_elenum+1)
    abundance_array = np.concatenate((abunarray1,abunarray2[1:]))
    return abundance_array

In [5]:
def abunprofile(ele,altarray,plawindex=-2):
    '''
    This creates a power law abundance profile for one element. Default value of power-law index is -2 for now, to be decided later.
    The profile looks like a(r) = a_0 * r^plawindex where r is altitude in planetary radius units. 
    '''
    zdict = tools.get_abundances() 
    ini_abundance = zdict[ele]
    radarray = altarray/altarray[0] #Distance array normalized with planetary radius
    abunarray = ini_abundance*(radarray**plawindex)
    return abunarray

In [6]:
def create_zdicts(fraclist,altarray):
    '''
    This function takes the elements that are fractionated and creates a list of dictionaries corresponding to abundances at 
    different altitudes. Profile is the variation of abundance (number not mass) with altitude, assumed to be same for all elements that are fractionated.
    '''
    frac_abundances = []
    #This loop gets the absolute (not relative to hydrogen) number (not mass) abundance profile for each fractionated element
    for ele in fraclist:
        frac_abundances.append(create_profile(ele,altarray))
    zdictlist = []
    #This loop creates a list of dictionaries. 
    for i in range(len(altarray)):
        zelemdict = dict((ele,abun[i]) for (ele,abun) in zip(fraclist,frac_abundances)) 
        #This gets abundance of each fractionated element relative to solar at a particular altitude and stores in a dictionary
        #This dictionary serves as the zelem argument
        zdictlist.append(get_zdict(zelem=zelemdict))
    return zdictlist

In [7]:
def calc_fracmubar(zdictlist):
    '''
    This function gets the initial neutral average mean molecular weight for a fractionated atmosphere.
    It returns both the mean molecular weight across the entire atmosphere, as well as the mean molecular weight varying with
    altitude.
    '''
    mulist = []
    for zdict in zdictlist:
        mulist.append(calc_neutral_mu(zdict))
    frac_mubar = np.mean(mulist)
    return frac_mubar, mulist